In [1]:
import os
import json
import math
import numpy as np
import matplotlib.pyplot as plt

def find_factors_x_ge_y(z):
    '''
    Find two factors of z such that x >= y, x * y = z and x should be as close as possible to y.
    '''
    z = abs(z)
    if z == 0: return 0, 0
    
    # 從平方根開始向下尋找第一個因數
    candidate = math.isqrt(z)
    
    for i in range(candidate, 0, -1):
        if z % i == 0:
            small_factor = i
            large_factor = z // i
            
            # 因為我們是從平方根往下找，large_factor 一定會 >= small_factor
            # 依照題目要求 X >= Y
            x, y = large_factor, small_factor
            return x, y

def plot_regret_grid(results_list, regret_key, title_prefix, filename, color, plot_type='line', is_log=False):
    """
    繪製 5x3 的 Regret 網格圖，支援對數轉換。
    """
    num_exps = len(results_list)  # 這裡設定為 15
    rows, cols = find_factors_x_ge_y(num_exps)
    fig, axes = plt.subplots(rows, cols, figsize=(18, 22))
    axes = axes.flatten()

    for i in range(num_exps):
        ax = axes[i]
        try:
            # 取得原始數據
            raw_data = np.array(results_list[i][regret_key])
            
            # --- 核心邏輯：對數轉換 ---
            if is_log:
                # 使用 np.maximum 確保數值不低於 1e-10，再取 log10
                plot_data = np.log10(np.maximum(raw_data, 1e-10))
                ylabel = "Log10(Regret)"
            else:
                plot_data = raw_data
                ylabel = "Regret Value"
            
            iters = range(1, len(plot_data) + 1)
            
            # 繪圖樣式
            if plot_type == 'line':
                ax.plot(iters, plot_data, color=color, marker='o', 
                        markersize=3, linewidth=1.2, alpha=0.8)
            else:
                ax.step(iters, plot_data, color=color, where='post', linewidth=2.5)

            # 標題優化
            file_name = os.path.basename(results_list[i].get('dataset_path', f'Exp_{i+1}'))
            ax.set_title(f"Trial {i+1}: {file_name}", fontsize=10, fontweight='bold')
            
            # 只有在非對數模式下才畫 y=0 的線 (對數模式下 y=0 代表原始值為 1)
            if not is_log:
                ax.axhline(0, color='black', linewidth=1, linestyle='--')
            
            ax.grid(True, which='both', linestyle=':', alpha=0.6)
            
            # 座標軸標籤
            if i % cols == 0: ax.set_ylabel(ylabel)
            if i >= (rows - 1) * cols: ax.set_xlabel('Iterations')

        except Exception as e:
            ax.set_title(f"Error in Trial {i+1}")
            print(f"Error: {e}")

    # 隱藏多餘子圖
    for j in range(num_exps, len(axes)):
        axes[j].axis('off')

    # 總標題調整
    log_status = "(Log Scale)" if is_log else "(Linear Scale)"
    fig.suptitle(f"{title_prefix} {log_status}", fontsize=18, y=1.02)
    fig.suptitle(f"Comparison of {title_prefix} across {num_exps} Trials", fontsize=16, y=1.02)

    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"成功儲存圖表：{filename}")
    plt.close(fig)

In [2]:
# 繪製 train_x 與 gt_x 距離的趨勢圖
def plot_distance_trend(results_list, filename, color='teal'):
    '''
    繪製 train_x 與 gt_x 距離的趨勢圖。
    '''
    num_exps = len(results_list)
    rows, cols = find_factors_x_ge_y(num_exps)
    fig, axes = plt.subplots(rows, cols, figsize=(18, 22))
    axes = axes.flatten()

    for i in range(num_exps):
        ax = axes[i]
        try:
            n_iter = len(results_list[i]['simple_regrets']) # 計算這次實驗跑了幾個 iteration
            gt_x = np.array(results_list[i]['gt_x']) # 取得 ground truth x
            train_x = np.array(results_list[i]['best_predicted_xs']) # 取得對應 iteration 的 train_x
            distances = np.sqrt(np.sum(np.square(train_x - gt_x), axis=1)) # 計算每個 iteration 推薦的 X 與 gt_x 的距離

            iters = range(1, n_iter + 1)

            ax.plot(iters, distances, color=color, marker='o', 
                    markersize=3, linewidth=1.2, alpha=0.8)

            file_name = os.path.basename(results_list[i].get('dataset_path', f'Exp_{i+1}'))
            ax.set_title(f"Trial {i+1}: {file_name}", fontsize=10, fontweight='bold')
            ax.set_ylabel("Distance")
            ax.set_xlabel("Iterations")
            ax.grid(True, which='both', linestyle=':', alpha=0.6)

        except Exception as e:
            ax.set_title(f"Error in Trial {i+1}")
            print(f"Error: {e}")

    for j in range(num_exps, len(axes)):
        axes[j].axis('off')

    fig.suptitle("Train X to GT X Distance Trend", fontsize=18, y=1.02)
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"成功儲存圖表：{filename}")
    plt.close(fig)

In [3]:
# result_path = '/workspaces/BO_EXPERIMENTS/src/results/20260120/D=10_N=5_K=6/BO_GP_EI_Matern_SLSQP_ITER100.json'
# output_plot_dir = '/workspaces/BO_EXPERIMENTS/src/results/20260120/D=10_N=5_K=6/'

In [4]:
result_dir = '/workspaces/BO_EXPERIMENTS/src/results/20260121/D=3_N=5_K=3/saasbo/jsonfiles'
output_plot_dir = '/workspaces/BO_EXPERIMENTS/src/results/20260121/D=3_N=5_K=3/saasbo/plots/'

In [ ]:
# 讀取 result.json 檔案
json_name_ls = os.listdir(result_dir)
json_path_ls = [os.path.join(result_dir, json_name) for json_name in json_name_ls if json_name.endswith('.json')]
results_list = []
for json_path in json_path_ls:
    with open(json_path, 'r') as f:
        result = json.load(f)
        results_list.append(result)

# 創建圖片儲存位置
os.makedirs(output_plot_dir, exist_ok=True)

In [ ]:
# 繪製 inference regrets 圖表
output_plot_name = 'inference_regret_log_plot.png'
output_plot_path = os.path.join(output_plot_dir, output_plot_name)
plot_regret_grid(
    results_list=results_list,
    regret_key='inference_regrets',
    title_prefix='Log Inference Regret',
    filename=output_plot_path,
    color='midnightblue',  # 你要求的深一點的顏色
    plot_type='line',
    is_log=True
)

# 繪製 simple regrets 圖表
output_plot_name = 'simple_regret_log_plot.png'
output_plot_path = os.path.join(output_plot_dir, output_plot_name)  
plot_regret_grid(
    results_list=results_list,
    regret_key='simple_regrets',
    title_prefix='Log Simple Regret',
    filename=output_plot_path,
    color='firebrick',  # 你要求的深一點的顏色
    plot_type='line',
    is_log=True
)

# 繪製 candidate_x 與 gt_x 的距離趨勢圖
output_plot_name = 'x_distance_plot.png'
output_plot_path = os.path.join(output_plot_dir, output_plot_name)  
plot_distance_trend(results_list, output_plot_path, color='teal')